In [1]:
# CS492H 2021 Prof. M Sung
# PA2 AutoEncoder
# Jeonghyun Kim

import os
import h5py
import easydict
import numpy as np
import torch
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from model import PointNet
from dataset import Dataset

# @Ref: https://github.com/chrdiller/pyTorchChamferDistance
from chamfer_distance.chamfer_distance import ChamferDistance 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [2]:
# Set Hyperparameters
args = easydict.EasyDict({
    'n_points': 2048,
    'latent': 512,
    'train': True, 
    'batch_size': 32,
    'epoch': 50,
    'n_workers': 4,
    'lr': 0.001,
    'b1': 0.9,
    'b2': 0.999,
    'step_size': 20,
    'gamma': 0.5,
    'data': 'shapenetcorev2',
    'model': '', #'outputs/model128_50.pth',
    'out_dir': 'outputs'
})

In [3]:
TRAIN_DATASET = Dataset(dataset_name=args.data, num_points=args.n_points, split='train')
TEST_DATASET = Dataset(dataset_name=args.data, num_points=args.n_points, split='test')
EVAL_DATASET = Dataset(dataset_name=args.data, num_points=args.n_points, split='val')

TRAIN_DATALOADER = torch.utils.data.DataLoader(TRAIN_DATASET, 
                                                   batch_size=args.batch_size,
                                                   shuffle=args.train,
                                                   num_workers=int(args.n_workers))
TEST_DATALOADER = torch.utils.data.DataLoader(TEST_DATASET,
                                                  batch_size=args.batch_size,
                                                  shuffle=args.train,
                                                  num_workers=int(args.n_workers)) 

In [4]:
# Set Network
n_dims = 3
net = PointNet(args.latent, n_dims, args.n_points)
net.to(device)

# Load model if there is any    
if args.model != '':
    net.load_state_dict(torch.load(args.model))

# Set Loss function
chamfer_distance = ChamferDistance()
criterion = chamfer_distance

# Set optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=args.lr, betas=(args.b1, args.b2))

# Set Scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)

In [5]:
def train_one_epoch(epoch=None, writer=None, epoch_str=None):    
    # Create a Progress bar.
    n = len(TRAIN_DATASET)
    pbar = tqdm(total=n, leave=False)
    
    # Train
    total_loss = 0.0
    net.train()
    for i, data in enumerate(TRAIN_DATALOADER):
        # Parse data
        points, label, category, filename = data
        points = points.cuda()
        
        # Reset gradients
        optimizer.zero_grad()
        
        # Reconstruction
        recon_ret, _ = net(points)
        
        # Compute Loss
        dist1, dist2 = criterion(points, recon_ret)
        loss = (torch.mean(dist1)) + (torch.mean(dist2))
        
        # Update Parameters
        loss.backward()
        optimizer.step()

        # Write results
        if writer is not None:
            assert(epoch is not None)
            step = epoch * len(TRAIN_DATALOADER) + i
            writer.add_scalar('Loss/Train', loss, step)
            
        batch_size = list(data[0].size())[0]
        total_loss += loss * batch_size
        
        pbar.set_description('{} Train Loss: {:f}'.format(epoch_str, loss))
        pbar.update(batch_size)
    
    # Close progress bar
    pbar.close()
    
    mean_loss = total_loss / n
    
    return mean_loss

In [6]:
from plyfile import PlyData, PlyElement

def dump_result(points, filename):
    """
        points: (N, 3)
        filename: str
    """
    points = [(points[i,0], points[i,1], points[i,2]) for i in range(points.shape[0])]
    vertex = np.array(points, dtype=[('x', 'f4'), ('y', 'f4'),('z', 'f4')])
    el = PlyElement.describe(vertex, 'vertex', comments=['vertices'])
    PlyData([el], text=True).write(filename)

In [7]:
def eval_one_epoch(epoch_str=None, dump=False):    
    n = len(TEST_DATASET)
    pbar = tqdm(total=n, leave=False)
    
    # Evaluate
    total_loss = 0.0
    net.eval()
    for i, data in enumerate(TEST_DATALOADER):
        # Parse data
        points, label, category, filename = data
        points = points.cuda()
        
        with torch.no_grad():
            recon_ret, _ = net(points)
            dist1, dist2 = criterion(points, recon_ret)
            loss = (torch.mean(dist1)) + (torch.mean(dist2))
            
        batch_size = list(data[0].size())[0]
        total_loss += loss * batch_size
        
        pbar.set_description('{} Train Loss: {:f}'.format(epoch_str, loss))
        pbar.update(batch_size)
        
        if dump:
            dump_result(points[0, :, :], './result{}/{}_points.ply'.format(args.latent, i))
            dump_result(recon_ret[0, :, :], './result{}/{}_recon.ply'.format(args.latent, i))
        
    pbar.close()
    
    mean_loss = total_loss / n
    
    return mean_loss

In [8]:
def train(n_epoch):    
    writer = SummaryWriter(args.out_dir)
    
    for epoch in range(n_epoch):
        epoch_str = '' if epoch is None else '[Epoch {}/{}]'.format(str(epoch).zfill(len(str(args.epoch))), args.epoch)
        
        # Compute Loss
        train_loss = train_one_epoch(epoch=epoch,
                                     writer=writer,
                                     epoch_str=epoch_str)
        test_loss = eval_one_epoch(epoch_str=epoch_str)
        
        # Update Scheduler
        scheduler.step()
        
        if writer is not None:
            # Write test results.
            assert(epoch is not None)
            step = (epoch + 1) * len(TRAIN_DATALOADER)
            writer.add_scalar('Loss/Test', test_loss, step)
        
        # Log statistics
        log = epoch_str + ' '
        log += 'Train Loss: {:f}, '.format(train_loss)
        log += 'Test Loss: {:f}, '.format(test_loss)
        print(log)
        
        # Save the model every 10 epochs
        if (epoch + 1) % 10 == 0:
            model_file = os.path.join(args.out_dir, 'model{}_{:d}.pth'.format(args.latent, epoch + 1))
            torch.save(net.state_dict(), model_file)
            print("Saved '{}'.".format(model_file))
        
    writer.close()

In [9]:
def eval():
    # Compute Loss
    test_loss = eval_one_epoch(dump=True)
    
    # Log statistics
    log = ''
    log += 'Avg Chamfer Distance: {:f} for {}'.format(test_loss, args.model)
    print(log)

In [10]:
if __name__ == "__main__":
    print(args)
    
    # Create the output directory.
    if not os.path.exists(args.out_dir):
        os.makedirs(args.out_dir)
        
    if args.train:
        train(args.epoch)
    else:
        eval()

  0%|          | 0/35708 [00:00<?, ?it/s][Epoch 08/50] Train Loss: 0.003641, Test Loss: 0.003913, 
[Epoch 09/50] Train Loss: 0.003467, Test Loss: 0.003432, 
  0%|          | 0/35708 [00:00<?, ?it/s][Epoch 18/50] Train Loss: 0.002996, Test Loss: 0.003034, 
[Epoch 19/50] Train Loss: 0.002789, Test Loss: 0.002962, 
  0%|          | 0/35708 [00:00<?, ?it/s][Epoch 28/50] Train Loss: 0.002337, Test Loss: 0.002667, 
[Epoch 29/50] Train Loss: 0.002332, Test Loss: 0.002600, 
  0%|          | 0/35708 [00:00<?, ?it/s][Epoch 38/50] Train Loss: 0.002212, Test Loss: 0.002595, 
[Epoch 39/50] Train Loss: 0.002186, Test Loss: 0.002610, 
  0%|          | 0/35708 [00:00<?, ?it/s][Epoch 48/50] Train Loss: 0.002024, Test Loss: 0.002458, 
[Epoch 49/50] Train Loss: 0.002010, Test Loss: 0.002389, 
Saved 'outputs/model512_50.pth'.
